This notebook performs profile normalization.
All profiles are normalized to the DMSO control treated profiles.

In [2]:
import argparse
import pathlib

import numpy as np
import pandas as pd
from pycytominer import normalize

# Get the current working directory
cwd = pathlib.Path.cwd()

if (cwd / ".git").is_dir():
    root_dir = cwd

else:
    root_dir = None
    for parent in cwd.parents:
        if (parent / ".git").is_dir():
            root_dir = parent
            break

# Check if a Git root directory was found
if root_dir is None:
    raise FileNotFoundError("No Git root directory found.")
try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False

In [3]:
if not in_notebook:
    argparser = argparse.ArgumentParser()
    argparser.add_argument(
        "--patient",
        type=str,
        required=True,
        help="Patient ID to process, e.g. 'P01'",
    )
    args = argparser.parse_args()
    patient = args.patient

else:
    patient = "NF0014"

In [4]:
# pathing
sc_annotated_path = pathlib.Path(
    f"{root_dir}/data/{patient}/image_based_profiles/2.annotated_profiles/sc_anno.parquet"
).resolve(strict=True)
organoid_annotated_path = pathlib.Path(
    f"{root_dir}/data/{patient}/image_based_profiles/2.annotated_profiles/organoid_anno.parquet"
).resolve(strict=True)


# output path
sc_normalized_output_path = pathlib.Path(
    f"{root_dir}/data/{patient}/image_based_profiles/3.normalized_profiles/sc_norm.parquet"
).resolve()
organoid_normalized_output_path = pathlib.Path(
    f"{root_dir}/data/{patient}/image_based_profiles/3.normalized_profiles/organoid_norm.parquet"
).resolve()

organoid_normalized_output_path.parent.mkdir(parents=True, exist_ok=True)

In [5]:
# read in the data
sc_annotated_profiles = pd.read_parquet(sc_annotated_path)
organoid_annotated_profiles = pd.read_parquet(organoid_annotated_path)

### Normalize the single-cell profiles

In [6]:
sc_metadata_columns = [
    "patient",
    "object_id",
    "unit",
    "dose",
    "treatment",
    "Target",
    "Class",
    "Therapeutic Categories",
    "image_set",
    "Well",
    "parent_organoid",
    "Area.Size.Shape_Cell_CENTER.X",
    "Area.Size.Shape_Cell_CENTER.Y",
    "Area.Size.Shape_Cell_CENTER.Z",
]
sc_features_columns = [
    col for col in sc_annotated_profiles.columns if col not in sc_metadata_columns
]

In [7]:
# find inf values and replace with NaN
sc_annotated_profiles[sc_features_columns] = sc_annotated_profiles[
    sc_features_columns
].replace([float("inf"), -float("inf")], np.nan)

In [ ]:
# normalize the data
sc_normalized_profiles = normalize(
    sc_annotated_profiles,
    features=sc_features_columns,
    meta_features=sc_metadata_columns,
    method="standardize",
    samples="treatment == 'DMSO'",
)
sc_normalized_profiles.to_parquet(sc_normalized_output_path, index=False)
sc_normalized_profiles.head()

,patient,object_id,unit,dose,treatment,Target,Class,Therapeutic Categories,image_set,Well,parent_organoid,Area.Size.Shape_Cell_CENTER.X,Area.Size.Shape_Cell_CENTER.Y,Area.Size.Shape_Cell_CENTER.Z,Area.Size.Shape_Nuclei_VOLUME,Area.Size.Shape_Nuclei_CENTER.X,Area.Size.Shape_Nuclei_CENTER.Y,Area.Size.Shape_Nuclei_CENTER.Z,Area.Size.Shape_Nuclei_BBOX.VOLUME,Area.Size.Shape_Nuclei_MIN.X,Area.Size.Shape_Nuclei_MAX.X,Area.Size.Shape_Nuclei_MIN.Y,Area.Size.Shape_Nuclei_MAX.Y,Area.Size.Shape_Nuclei_MIN.Z,Area.Size.Shape_Nuclei_MAX.Z,Area.Size.Shape_Nuclei_EXTENT,Area.Size.Shape_Nuclei_EULER.NUMBER,Area.Size.Shape_Nuclei_EQUIVALENT.DIAMETER,Area.Size.Shape_Nuclei_SURFACE.AREA,Colocalization_Nuclei_AGP.ER_MEAN.CORRELATION.COEFF,Colocalization_Nuclei_AGP.ER_MEDIAN.CORRELATION.COEFF,Colocalization_Nuclei_AGP.ER_MIN.CORRELATION.COEFF,Colocalization_Nuclei_AGP.ER_MAX.CORRELATION.COEFF,Colocalization_Nuclei_AGP.ER_MEAN.MANDERS.COEFF.M1,Colocalization_Nuclei_AGP.ER_MEDIAN.MANDERS.COEFF.M1,Colocalization_Nuclei_AGP.ER_MIN.MANDERS.COEFF.M1,Colocalization_Nuclei_AGP.ER_MAX.MANDERS.COEFF.M1,Colocalization_Nuclei_AGP.ER_MEAN.MANDERS.COEFF.M2,Colocalization_Nuclei_AGP.ER_MEDIAN.MANDERS.COEFF.M2,Colocalization_Nuclei_AGP.ER_MIN.MANDERS.COEFF.M2,Colocalization_Nuclei_AGP.ER_MAX.MANDERS.COEFF.M2,Colocalization_Nuclei_AGP.ER_MEAN.OVERLAP.COEFF,Colocalization_Nuclei_AGP.ER_MEDIAN.OVERLAP.COEFF,Colocalization_Nuclei_AGP.ER_MIN.OVERLAP.COEFF,Colocalization_Nuclei_AGP.ER_MAX.OVERLAP.COEFF,Colocalization_Nuclei_AGP.ER_MEAN.K1,Colocalization_Nuclei_AGP.ER_MEDIAN.K1,Colocalization_Nuclei_AGP.ER_MIN.K1,Colocalization_Nuclei_AGP.ER_MAX.K1,Colocalization_Nuclei_AGP.ER_MEAN.K2,...,Texture_Cytoplasm_AGP_Correlation_256.3,Texture_Cytoplasm_AGP_Difference.Entropy_256.3,Texture_Cytoplasm_AGP_Difference.Variance_256.3,Texture_Cytoplasm_AGP_Entropy_256.3,Texture_Cytoplasm_AGP_Information.Measure.of.Correlation.1_256.3,Texture_Cytoplasm_AGP_Information.Measure.of.Correlation.2_256.3,Texture_Cytoplasm_AGP_Inverse.Difference.Moment_256.3,Texture_Cytoplasm_AGP_Sum.Average_256.3,Texture_Cytoplasm_AGP_Sum.Entropy_256.3,Texture_Cytoplasm_AGP_Sum.Variance_256.3,Texture_Cytoplasm_AGP_Variance_256.3,Texture_Cytoplasm_DNA_Angular.Second.Moment_256.3,Texture_Cytoplasm_DNA_Contrast_256.3,Texture_Cytoplasm_DNA_Correlation_256.3,Texture_Cytoplasm_DNA_Difference.Entropy_256.3,Texture_Cytoplasm_DNA_Difference.Variance_256.3,Texture_Cytoplasm_DNA_Entropy_256.3,Texture_Cytoplasm_DNA_Information.Measure.of.Correlation.1_256.3,Texture_Cytoplasm_DNA_Information.Measure.of.Correlation.2_256.3,Texture_Cytoplasm_DNA_Inverse.Difference.Moment_256.3,Texture_Cytoplasm_DNA_Sum.Average_256.3,Texture_Cytoplasm_DNA_Sum.Entropy_256.3,Texture_Cytoplasm_DNA_Sum.Variance_256.3,Texture_Cytoplasm_DNA_Variance_256.3,Texture_Cytoplasm_ER_Angular.Second.Moment_256.3,Texture_Cytoplasm_ER_Contrast_256.3,Texture_Cytoplasm_ER_Correlation_256.3,Texture_Cytoplasm_ER_Difference.Entropy_256.3,Texture_Cytoplasm_ER_Difference.Variance_256.3,Texture_Cytoplasm_ER_Entropy_256.3,Texture_Cytoplasm_ER_Information.Measure.of.Correlation.1_256.3,Texture_Cytoplasm_ER_Information.Measure.of.Correlation.2_256.3,Texture_Cytoplasm_ER_Inverse.Difference.Moment_256.3,Texture_Cytoplasm_ER_Sum.Average_256.3,Texture_Cytoplasm_ER_Sum.Entropy_256.3,Texture_Cytoplasm_ER_Sum.Variance_256.3,Texture_Cytoplasm_ER_Variance_256.3,Texture_Cytoplasm_Mito_Angular.Second.Moment_256.3,Texture_Cytoplasm_Mito_Contrast_256.3,Texture_Cytoplasm_Mito_Correlation_256.3,Texture_Cytoplasm_Mito_Difference.Entropy_256.3,Texture_Cytoplasm_Mito_Difference.Variance_256.3,Texture_Cytoplasm_Mito_Entropy_256.3,Texture_Cytoplasm_Mito_Information.Measure.of.Correlation.1_256.3,Texture_Cytoplasm_Mito_Information.Measure.of.Correlation.2_256.3,Texture_Cytoplasm_Mito_Inverse.Difference.Moment_256.3,Texture_Cytoplasm_Mito_Sum.Average_256.3,Texture_Cytoplasm_Mito_Sum.Entropy_256.3,Texture_Cytoplasm_Mito_Sum.Variance_256.3,Texture_Cytoplasm_Mito_Variance_256.3
0,NF0014,25,uM

### Normalize the organoid profiles

In [8]:
organoid_annotated_profiles.head()

,patient,object_id,unit,dose,treatment,image_set,Well,single_cell_count,Area.Size.Shape_Organoid_VOLUME,Area.Size.Shape_Organoid_CENTER.X,...,Texture_Organoid_Mito_Information.Measure.of.Correlation.1_256.3,Texture_Organoid_Mito_Information.Measure.of.Correlation.2_256.3,Texture_Organoid_Mito_Inverse.Difference.Moment_256.3,Texture_Organoid_Mito_Sum.Average_256.3,Texture_Organoid_Mito_Sum.Entropy_256.3,Texture_Organoid_Mito_Sum.Variance_256.3,Texture_Organoid_Mito_Variance_256.3,Target,Class,Therapeutic Categories
0,NF0014,40,uM,10,Mirdametinib,G8-1,G8,8.0,6416256.0,769.689636,...,-0.538078,0.720674,0.943387,5.153550,0.835848,479.284825,121.910734,MEK1/2 inhibitor,Small Molecule,Kinase Inhibitor
1,NF0014,24,uM,1,Fimepinostat,D5-1,D5,5.0,3727848.0,904.163208,...,-0.198021,0.423806,0.792245,4.166492,1.731223,266.549623,92.589375,PI3K and HDAC inhibitor,Small Molecule,Investigational
2,NF0014,85,uM,1,Mirdametinib,F8-1,F8,9.0,19925580.0,609.403259,...,-0.416777,0.919123,0.758222,9.480620,2.757967,167.525981,43.794476,MEK1/2 inhibitor,Small Molecule,Kinase Inhibitor
3,NF0014,18,nM,10,Staurosporine,G11-1,G11,1.0,2721536.0,985.260254,...,-0.577786,0.694415,0.947585,6.919087,0.700396,862.401508,221.828718,Apoptosis induction,Small Molecule,Experimental
4,NF0014,24,uM,1,Binimetinib,G7-1,G7,7.0,6343511.0,925.204590,...,-0.525013,0.819592,0.903090,14.716854,1.289324,2075.415990,527.099865,MEK1/2 inhibitor,Small Molecule,Kinase Inhibitor


In [11]:
organoid_metadata_columns = [
    "patient",
    "object_id",
    "unit",
    "dose",
    "treatment",
    "Target",
    "Class",
    "Therapeutic Categories",
    "image_set",
    "Well",
    "single_cell_count",
    "Area.Size.Shape_Organoid_CENTER.X",
    "Area.Size.Shape_Organoid_CENTER.Y",
    "Area.Size.Shape_Organoid_CENTER.Z",
]
organoid_features_columns = [
    col
    for col in organoid_annotated_profiles.columns
    if col not in organoid_metadata_columns
]
# normalize the data
organoid_normalized_profiles = normalize(
    organoid_annotated_profiles,
    features=organoid_features_columns,
    meta_features=organoid_metadata_columns,
    method="standardize",
    samples="treatment == 'DMSO'",
)
organoid_normalized_profiles.to_parquet(organoid_normalized_output_path, index=False)
organoid_normalized_profiles.head()

,patient,object_id,unit,dose,treatment,Target,Class,Therapeutic Categories,image_set,Well,single_cell_count,Area.Size.Shape_Organoid_CENTER.X,Area.Size.Shape_Organoid_CENTER.Y,Area.Size.Shape_Organoid_CENTER.Z,Area.Size.Shape_Organoid_VOLUME,Area.Size.Shape_Organoid_BBOX.VOLUME,Area.Size.Shape_Organoid_MIN.X,Area.Size.Shape_Organoid_MAX.X,Area.Size.Shape_Organoid_MIN.Y,Area.Size.Shape_Organoid_MAX.Y,Area.Size.Shape_Organoid_MIN.Z,Area.Size.Shape_Organoid_MAX.Z,Area.Size.Shape_Organoid_EXTENT,Area.Size.Shape_Organoid_EULER.NUMBER,Area.Size.Shape_Organoid_EQUIVALENT.DIAMETER,Area.Size.Shape_Organoid_SURFACE.AREA,Colocalization_Organoid_AGP.ER_MEAN.CORRELATION.COEFF,Colocalization_Organoid_AGP.ER_MEDIAN.CORRELATION.COEFF,Colocalization_Organoid_AGP.ER_MIN.CORRELATION.COEFF,Colocalization_Organoid_AGP.ER_MAX.CORRELATION.COEFF,Colocalization_Organoid_AGP.ER_MEAN.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.ER_MEDIAN.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.ER_MIN.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.ER_MAX.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.ER_MEAN.MANDERS.COEFF.M2,Colocalization_Organoid_AGP.ER_MEDIAN.MANDERS.COEFF.M2,Colocalization_Organoid_AGP.ER_MIN.MANDERS.COEFF.M2,Colocalization_Organoid_AGP.ER_MAX.MANDERS.COEFF.M2,Colocalization_Organoid_AGP.ER_MEAN.OVERLAP.COEFF,Colocalization_Organoid_AGP.ER_MEDIAN.OVERLAP.COEFF,Colocalization_Organoid_AGP.ER_MIN.OVERLAP.COEFF,Colocalization_Organoid_AGP.ER_MAX.OVERLAP.COEFF,Colocalization_Organoid_AGP.ER_MEAN.K1,Colocalization_Organoid_AGP.ER_MEDIAN.K1,Colocalization_Organoid_AGP.ER_MIN.K1,Colocalization_Organoid_AGP.ER_MAX.K1,Colocalization_Organoid_AGP.ER_MEAN.K2,Colocalization_Organoid_AGP.ER_MEDIAN.K2,Colocalization_Organoid_AGP.ER_MIN.K2,Colocalization_Organoid_AGP.ER_MAX.K2,...,Texture_Organoid_AGP_Correlation_256.3,Texture_Organoid_AGP_Difference.Entropy_256.3,Texture_Organoid_AGP_Difference.Variance_256.3,Texture_Organoid_AGP_Entropy_256.3,Texture_Organoid_AGP_Information.Measure.of.Correlation.1_256.3,Texture_Organoid_AGP_Information.Measure.of.Correlation.2_256.3,Texture_Organoid_AGP_Inverse.Difference.Moment_256.3,Texture_Organoid_AGP_Sum.Average_256.3,Texture_Organoid_AGP_Sum.Entropy_256.3,Texture_Organoid_AGP_Sum.Variance_256.3,Texture_Organoid_AGP_Variance_256.3,Texture_Organoid_DNA_Angular.Second.Moment_256.3,Texture_Organoid_DNA_Contrast_256.3,Texture_Organoid_DNA_Correlation_256.3,Texture_Organoid_DNA_Difference.Entropy_256.3,Texture_Organoid_DNA_Difference.Variance_256.3,Texture_Organoid_DNA_Entropy_256.3,Texture_Organoid_DNA_Information.Measure.of.Correlation.1_256.3,Texture_Organoid_DNA_Information.Measure.of.Correlation.2_256.3,Texture_Organoid_DNA_Inverse.Difference.Moment_256.3,Texture_Organoid_DNA_Sum.Average_256.3,Texture_Organoid_DNA_Sum.Entropy_256.3,Texture_Organoid_DNA_Sum.Variance_256.3,Texture_Organoid_DNA_Variance_256.3,Texture_Organoid_ER_Angular.Second.Moment_256.3,Texture_Organoid_ER_Contrast_256.3,Texture_Organoid_ER_Correlation_256.3,Texture_Organoid_ER_Difference.Entropy_256.3,Texture_Organoid_ER_Difference.Variance_256.3,Texture_Organoid_ER_Entropy_256.3,Texture_Organoid_ER_Information.Measure.of.Correlation.1_256.3,Texture_Organoid_ER_Information.Measure.of.Correlation.2_256.3,Texture_Organoid_ER_Inverse.Difference.Moment_256.3,Texture_Organoid_ER_Sum.Average_256.3,Texture_Organoid_ER_Sum.Entropy_256.3,Texture_Organoid_ER_Sum.Variance_256.3,Texture_Organoid_ER_Variance_256.3,Texture_Organoid_Mito_Angular.Second.Moment_256.3,Texture_Organoid_Mito_Contrast_256.3,Texture_Organoid_Mito_Correlation_256.3,Texture_Organoid_Mito_Difference.Entropy_256.3,Texture_Organoid_Mito_Difference.Variance_256.3,Texture_Organoid_Mito_Entropy_256.3,Texture_Organoid_Mito_Information.Measure.of.Correlation.1_256.3,Texture_Organoid_Mito_Information.Measure.of.Correlation.2_256.3,Texture_Organoid_Mito_Inverse.Difference.Moment_256.3,Texture_Organoid_Mito_Sum.Average_256.3,Texture_Organoid_Mito_Sum.Entropy_256.3,Texture_Organoid_Mito_Sum.Variance_256.3,